# Welcome to your Database
## We will be Connecting Python code to a SQL Database
In this assignment we will be running a webform and will be adapting the code to use a database backend. Once we have sucessfully used a database backend we will then find and fix the most common bugs affecting webapp database and fixing the damage that they can cause to your database. Lets get started!

![](img/dbapi1.PNG)

In the first two notebooks we have gone though how to get a database to give up and output given a query and had used a black box to hide the bits about how a query gets the information. If you were following the course on Udacity the querys you enter were send to the web servers but now we will go into more depth about how the magic happens.

![](img/dbapi2.PNG)

Behind the Udacity web servers is python code that allows us to connect to an sqlite database using the DB-API method calls. These DB_API isnt a library but a standard for libraries to left python code connect to databases.  If you learnt he BD-API functions you can apply those to any database system. Below we have Database Systems and their corresponding libraries.

![](img/dbapi3.PNG)

In the vm we will be using the PostgreSQL.

![](img/dbapi4.PNG)

Above is an example of using python code with the DB-API and in this case sqlite3. The general rules can be applied to any database with the DB-API. 
1. We want to do is import the DB-API
2. Connect to your Database
3. initialize  our cursor 